In [4]:
import sys
import networkx as nx
import ndlib.models.epidemics as ep
from bokeh.io import output_file, save
from bokeh.plotting import figure
import ndlib.models.ModelConfig as mc
import os

try:
    # Get parameters from command line arguments
    beta = float(sys.argv[1])
    gamma = float(sys.argv[2])

    # Print only the received parameters
    print(f"Parameters received - beta: {beta}, gamma: {gamma}")

    # Network Definition
    g = nx.erdos_renyi_graph(1000, 0.1)

    # Model Selection
    model = ep.SIRModel(g)

    # Model Configuration
    config = mc.Configuration()
    config.add_model_parameter('beta', beta)
    config.add_model_parameter('gamma', gamma)
    model.set_initial_status(config)

    # Simulation
    iterations = model.iteration_bunch(200)
    trends = model.build_trends(iterations)

    # Ensure the plot directory exists
    plot_directory = os.path.join(os.path.dirname(os.path.abspath(__file__)), "../plot")
    os.makedirs(plot_directory, exist_ok=True)

    # Manual Plotting
    output_file(os.path.join(plot_directory, "plot.html"))
    p = figure(title="SIR Model Simulation", x_axis_label='Iteration', y_axis_label='Number of Nodes')

    # Extracting data for plotting
    if trends and 'trends' in trends[0] and 'node_count' in trends[0]['trends']:
        trends_data = trends[0]['trends']
        susceptible = trends_data['node_count'][0]
        infected = trends_data['node_count'][1]
        removed = trends_data['node_count'][2]

        p.line(list(range(len(susceptible))), susceptible, color='blue', legend_label='Susceptible')
        p.line(list(range(len(infected))), infected, color='green', legend_label='Infected')
        p.line(list(range(len(removed))), removed, color='red', legend_label='Removed')
    else:
        raise ValueError("Trend data is not in the expected format")

    save(p)
except Exception as e:
    print(f"An error occurred: {e}")
    sys.exit(1)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occurred: could not convert string to float: '-f'
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_14608\2915542073.py", line 11, in <module>
    beta = float(sys.argv[1])
ValueError: could not convert string to float: '-f'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_14608\2915542073.py", line 57, in <module>
    sys.exit(1)
SystemExit: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\User\anaconda3\lib\site-packages\IPyth

TypeError: object of type 'NoneType' has no len()